In [1]:
import math
from math import sqrt
from sympy import *

# Pipeflow: Reynold's number

In [2]:
def iterative_moody(Re): 
    f = 0.02
    while True:
        right = 2.0*math.log10(Re*sqrt(f)) - 0.8
        f_new = (1/right)**2
        if abs(f_new - f) < 0.001:
            return f_new
        else:
            f = f_new

def Re(d_viscosity, density, k_viscosity, v, D, Q, dmdt):
    #trying to find values indirectly
    if d_viscosity == None:
        try:
            d_viscosity = k_viscosity*density
        except:
            pass
    elif density == None:
        try:
            density = d_viscosity/k_viscosity
        except:
            pass
    elif k_viscosity == None:
        try:
            k_viscosity = d_viscosity/density
        except:
            pass
    
    if dmdt:
        v = dmdt/(((math.pi*(D**2)/4))*density)
    elif Q:
        v = Q/((math.pi*(D**2)/4))

    #calculating Re number
    if k_viscosity == None:
        Re = density*v*D/d_viscosity
    else: 
        Re = v*D/k_viscosity
    
    print(f'Reynolds number is {Re}')
    
    if Re < 2300:
        print('flow is laminar')
        print(f'entrance length is: {D*0.06*Re}')
        f = 64/Re
        print(f'friction factor f is: {f}')
        try:
            print(f'wall shear stress is: {(f*density*v**2)/8}')
        except:
            pass
        length = float(input('please input length to get head loss: '))
        g = float(input('please input g: '))
        hl = f*(length/D)*(v**2)/(2*g)
        print(f'head loss is: {hl}')
        try:
            print(f'pressure drop is: {density*g*hl}')
        except:
            pass   
        return
        
    cont = True
    if Re > 2300 and Re < 10**5:
        print('flow is transitional')
        if Re > 2000 and Re < 4000:
            print('no reliable f value')
        cont = input('Do you want to consider as Turbulent? 1 or 0 ')
        
    while cont:
        if Re > 10**5:
            print('flow is turbulent')
            print(f'entrance length is: {D*4.4*(Re**(1/6))}')
        
        check = input('what is the hydraulic roughness? smooth, rough or transitional rough? Or you have f? ')
        if check == 'smooth':
            if Re > 4000 and Re < 10**5:
                f = 0.316*(Re**0.25)
                print(f'friction factor f is: {f}')
                try:
                    print(f'wall shear stress is: {(f*density*v**2)/8}')
                except:
                    pass
                length = float(input('please input length to get head loss: '))
                g = float(input('please input g: '))
                hl = f*(length/D)*(v**2)/(2*g)
                print(f'head loss is: {hl}')
                try:
                    print(f'pressure drop is: {density*g*hl}')
                except:
                    pass  
                
            else:
                f = iterative_moody(70174)
                f = 0.316*(Re**0.25)
                print(f'friction factor f is: {f}')
                try:
                    print(f'wall shear stress is: {(f*density*v**2)/8}')
                except:
                    pass
                length = float(input('please input length to get head loss: '))
                g = float(input('please input g: '))
                print(f'head loss is: {f*(length/D)*(v**2)/(2*g)}')
                if bool(density) == True:
                    length = float(input('please input length to get pressure  loss: '))
                    print(f'pressure drop estimate is: {0.158*length*(density**0.75)*(d_viscosity**0.25)*(D**-1.25)*(v**-1.75)}')
            return
        
        if check == 'rough':
            print('see moody diagram')
            rel_roughness = float(input('input relative roughness for estimate of f: '))
            f = (1/(-2*math.log10(rel_roughness/3.7)))**2
            print(f'approximate value of f is: {f}')
            try:
                print(f'wall shear stress is: {(f*density*v**2)/8}')
            except:
                pass
            length = float(input('please input length to get head loss: '))
            g = float(input('please input g: '))
            hl = f*(length/D)*(v**2)/(2*g)
            print(f'head loss is: {hl}')
            try:
                print(f'pressure drop is: {density*g*hl}')
            except:
                pass
            return
        
        if check == 'transitional rough':
            print('see moody diagram')
            rel_roughness = float(input('input relative roughness for estimate of f: '))
            f = (1/(-1.8*math.log10(6.9/Re+((rel_roughness)**1.11)/3.7)))**2
            print(f'approximate value of f is: {f}')
            try:
                print(f'wall shear stress is: {(f*density*v**2)/8}')
            except:
                pass
            length = float(input('please input length to get head loss: '))
            g = float(input('please input g: '))
            hl = f*(length/D)*(v**2)/(2*g)
            print(f'head loss is: {hl}')
            try:
                print(f'pressure drop is: {density*g*hl}')
            except:
                pass  
            return
        else:
            pass
        
        if check == 'f':
            f = float(input('please input f: '))
            try:
                wall_shear_stress = (f*density*v**2)/8
                print(f'wall shear stress is: {wall_shear_stress}')
                u_star = math.sqrt(wall_shear_stress/density)
                print(f'u_star is: {u_star}')
            except:
                pass
            length = float(input('please input length to get head loss: '))
            g = float(input('please input g: '))
            print(f'head loss is: {f*(length/D)*(v**2)/(2*g)}')
            hl = f*(length/D)*(v**2)/(2*g)
            print(f'head loss is: {hl}')
            try:
                print(f'pressure drop is: {density*g*hl}')
            except:
                pass  
            
            roughness = float(input('please input roughness to get pipe smoothness classification: '))
            try:
                val = roughness*u_star/k_viscosity
                if val < 5:
                    print('pipe is smooth')
                    umax = u_star * (2.5 * math.log(D*u_star/(2*roughness)) + 5)
                    print(f'umax is: {umax}')
                    print(f'average velocity is: {umax/(1+1.33*math.sqrt(f))}')
                elif val < 70:
                    print('pipe is transitional rough')
                else:
                    print('pipe is rough')
                    print(f'umax is: {u_star * (2.5 * math.log(D/(2*roughness)) + 8.5)}')
                    
            except:
                pass
            
            return
            
        
        

In [ ]:
d_viscosity = 0.005
density = 1000
k_viscosity = 1e-6
v = 4.223e-2
D = 6e-2
Q = None
dmdt = None

Re(d_viscosity, density, k_viscosity, v, D, Q, dmdt)

Reynolds number is 2533.7999999999997
flow is transitional
no reliable f value
Do you want to consider as Turbulent? 1 or 0 1


# Continuity Calculator

In [3]:
def continuity(D1, V1, D2, V2):
    if D2 == None:
        print(f'D2 is: { math.sqrt((V1 * (D1**2)/4)/V2)}')
    if V2 == None:
        print(f'V2 is: { V1 * ((D1/D2)**2)}')

In [ ]:
D1 = 0.01
V1 = 2

#unknowns
D2 = None
V2 = 4

continuity(D1, V1, D2, V2)

# Stability

In [5]:
x = Symbol("H")
y = Symbol("H")
z = Symbol("H") #into the plane
ratio = 0.7

KB = None
KG = 0.3*y #None
BM = None

h_unsub = (1- ratio)*y
h_sub = y-h_unsub
#h_sub = Symbol("H")
Vol_sub =  (h_sub)*x*z

KB = h_sub/2
BM = (z*(x**3)/12)/Vol_sub
#KG = y/2

print(f'KB is: {KG}')
print(f'KG is: {KG}')
print(f'BM is: {BM}')

print('GM is: ')
KB + BM - KG

KB is: 0.3*H
KG is: 0.3*H
BM is: 0.119047619047619*H
GM is: 


0.169047619047619*H